<a href="https://colab.research.google.com/github/zanzivyr/Optimizers/blob/main/SchurDecomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schur Decomposition

Using our study of the Gram-Schmidt Orthogonalization process, we can now perform a Schur Decomposition on a matrix, such that:

Given A, T = UᵀAU

We will first solve for U, and then multiply to arrive at T.

1. Find Eigenvectors
2. Apply Gram-Schmidt
3. Normalize (now orthonormal basis)
4. Multiply

## Application

We are specifically visiting Schur Decomposition for the sake of solving C.A.R.E. (Continuous Algebraic Riccati Equations), which are found in Control Theory when finding an optimal LQR controller.

# Resources

- Matt B. - https://youtu.be/Giz6hxC9VGI

In [1]:
import numpy as np
from numpy import linalg as LA
import tensorflow as tf

# Instantiate

Create a random square (n x n) matrix A.

In [2]:
n = 3

A = tf.random.uniform(shape=[n,n], maxval=10, dtype=tf.float32)
A.numpy()

array([[7.2352753, 9.983719 , 8.925456 ],
       [5.5139866, 6.0656586, 0.1578641],
       [4.7421384, 6.427475 , 4.0137205]], dtype=float32)

# 1. Find Eigenvectors

In [7]:
w, vset = LA.eig(A)
vectors = vset.shape[0]
size = vset.shape[1]
vset

array([[ 0.79265314+0.j        ,  0.72583103+0.j        ,
         0.72583103-0.j        ],
       [ 0.39116427+0.j        , -0.6415961 -0.10164887j,
        -0.6415961 +0.10164887j],
       [ 0.46764466+0.j        ,  0.12468033+0.1888019j ,
         0.12468033-0.1888019j ]], dtype=complex64)

# 2. Apply Gram-Schmidt

In [8]:
basis = vset[0]

for v in range(1, vectors):
  vec = vset[v]

  sum = 0
  for u in range(1, v+1):
    uvec = vset[u]
    sum += np.dot((np.dot(uvec, vec) / np.dot(uvec, uvec)), uvec)

  b = vec - sum
  basis = tf.concat([basis, b], 0)

basis = tf.reshape(basis, (vectors,size), name=None)
basis

<tf.Tensor: shape=(3, 3), dtype=complex64, numpy=
array([[ 0.79265314+0.j        ,  0.72583103+0.j        ,
         0.72583103-0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ],
       [-0.02509975+0.j        ,  0.04116916+0.00652248j,
         0.04116916-0.00652248j]], dtype=complex64)>

# 3. Normalize

The basis found through the Gram-Schmidt Process will now become orthonormal.

In [10]:
ortho = 0

if LA.norm(basis[0]) != 0:
  ortho = tf.math.multiply(basis[0], 1/LA.norm(basis[0]))
else:
  ortho = tf.zeros(size, dtype=tf.complex64)

for v in range(1, vectors):
  if LA.norm(basis[v]) != 0:
    o = tf.math.multiply(basis[v], 1/LA.norm(basis[v]))
  else:
    o = tf.zeros(size, dtype=tf.complex64)
    
  ortho = tf.concat([ortho, o], 0)

ortho = tf.reshape(ortho, (vectors,size), name=None)
ortho

<tf.Tensor: shape=(3, 3), dtype=complex64, numpy=
array([[ 0.6111887 +0.j        ,  0.55966437+0.j        ,
         0.55966437+0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ],
       [-0.3917591 +0.j        ,  0.64257175+0.10180337j,
         0.64257175-0.10180337j]], dtype=complex64)>

# 4. Multiply

Now with the orthonormal basis, multiply with the given A matrix to find T.

In [12]:
U = ortho
UT = np.transpose(U)

T = np.dot(UT, np.dot(A, U))
T

array([[0.046204 +0.j        , 3.9301016+0.39527464j,
        3.9301016-0.39527464j],
       [1.3699689+0.13498414j, 8.797126 +1.3038439j ,
        8.880322 -0.23834753j],
       [1.3699689-0.13498414j, 8.880322 +0.23834753j,
        8.797126 -1.3038439j ]], dtype=complex64)